# ONS Census 2021 - PII Risk

### Importing/Cleaning Data

### Combining Variables

### Identifying Select Cells

Function to highlight cells that are below the critical level of 10.

Data must be input in the form

| Region     | Population | Variable | ... |
|------------|------------|----------|-----|
| North East | 2647013    |  2536430 | ... |
| North West | 7417399    | 6885187  | ... |

and will be output in the form

| Region     | Population | Probability | ... |
|------------|------------|-------------|-----|
| North East | 2647013    |    0.958223 | ... |
| North West | 7417399    | 0.928248    | ... |

In [3]:
def highlight_critical_cells(dataframe):
    # Iterates through columns in dataframe
    for num in range(len(dataframe.columns)):
        column = dataframe.loc[:,dataframe.columns[count]]
        cell_count = 0
        # Iterates through items in each column
        for row in column:
            #Highlights cell based on size
            column.style.apply('background-color: blue')  
            cell_count += 1
        count += 1
    return dataframe

### Probability Function

Function to calculate probailities as a percent of the total in each region.

Data must be input in the form

| Region     | Population | Variable | ... |
|------------|------------|----------|-----|
| North East | 2647013    |  2536430 | ... |
| North West | 7417399    | 6885187  | ... |

and will be output in the form

| Region     | Population | Probability | ... |
|------------|------------|-------------|-----|
| North East | 2647013    |    0.958223 | ... |
| North West | 7417399    | 0.928248    | ... |  |  |  |

In [2]:
def prob_calc(dataframe):
    probabilities = pd.DataFrame(dataframe)
    total = dataframe.loc[:,dataframe.columns[1]]
    
    count = 2

    # Iterates through columns in dataframe
    for num in range(len(dataframe.columns)-2):
        column = dataframe.loc[:,dataframe.columns[count]]
        cell_count = 0
        # Iterates through items in each column
        for row in column:
            #Divides current cell by the item at the start of the row
            probabilities.loc[:, dataframe.columns[count]].at[cell_count] = row / total[cell_count]
            cell_count += 1
        count += 1
    
    return probabilities

### Correlations in the Data

### Visualising the Data

### Extension - Research how ONS privacy protection exercises have impacted the final results

### Extension - Explore wider utility of the census data for other Hartree Centre projects

### Extension - Consider other open data sources